In [7]:
# Import the required packages.

import numpy as np
import pandas as pd
import sklearn

In [8]:
# Load dataset

data = pd.read_csv(f'insurance.csv')

In [9]:
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [10]:
# Looking statistics data

data.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [11]:
# Create a dependent variable and independent variable

X = data.drop('charges', axis=1)
y = data['charges']

In [12]:
X.head()

,age,sex,bmi,children,smoker,region
0,19,female,27.900,0,yes,southwest
1,18,male,33.770,1,no,southeast
2,28,male,33.000,3,no,southeast
3,33,male,22.705,0,no,northwest
4,32,male,28.880,0,no,northwest


In [13]:
y.head()

0    16884.92400
1     1725.55230
2     4449.46200
3    21984.47061
4     3866.85520
Name: charges, dtype: float64

## Handling missing values

In [15]:
# Count the missing values in each column of dataset

print(data.isnull().sum())

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64


In [16]:
# Removing(drop) the missing values 
data_01 = data.dropna(inplace = True)

In [23]:
# Replace missing value without drop all records
from sklearn.impute import SimpleImputer

# Separate numerical and categorical columns
num_col = data.select_dtypes(include=['float64', 'int64']).columns
cat_col = data.select_dtypes(include=['object']).columns

# Impute numerical column with mean
num_imputer = SimpleImputer(strategy = 'mean')
data[num_col] = num_imputer.fit_transform(data[num_col])

# Impute categorical columns
cat_imputer = SimpleImputer(strategy = 'most_frequent')
data[cat_col] = cat_imputer.fit_transform(data[cat_col])

# Dsiplay the results 
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19.0,female,27.900,0.0,yes,southwest,16884.92400
1,18.0,male,33.770,1.0,no,southeast,1725.55230
2,28.0,male,33.000,3.0,no,southeast,4449.46200
3,33.0,male,22.705,0.0,no,northwest,21984.47061
4,32.0,male,28.880,0.0,no,northwest,3866.85520


## Data Encoding (Handle categorical data)

In [25]:
# One Hot encoding 

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Separate the independent variable (X)
X = data.drop('charges', axis = 1)

# Define the ColumnTransformer
ct = ColumnTransformer(
    transformers = [
        ('encoder', OneHotEncoder(), ['sex', 'smoker', 'region'])
    ],
    remainder = 'passthrough' # Leave the other column unchanged
)

# Apply the ColumnTransformer
X_transformed = ct.fit_transform(X)

# Get feature names for the one-hot encoded columns
encoded_feature_names = ct.named_transformers_['encoder'].get_feature_names_out(['sex', 'smoker', 'region'])

# Combine encoded feature names with the remainder column names
all_feature_names = np.concatenate([encoded_feature_names, X.columns.difference(['sex', 'smoker', 'region'])])

# Convert the transformed data into a DataFrame
X_transformed_df = pd.DataFrame(X_transformed, columns=all_feature_names)

# Display the resulting DataFrame
X_transformed_df.head()

,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest,age,bmi,children
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,19.0,27.900,0.0
1,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,18.0,33.770,1.0
2,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,28.0,33.000,3.0
3,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,33.0,22.705,0.0
4,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,32.0,28.880,0.0


In [27]:
# Label encoding 

from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
le = LabelEncoder()

# Perform label encoding
data['smoker'] = le.fit_transform(data['smoker'])

# Display the Dataframe with original and encoded columns
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19.0,female,27.900,0.0,1,southwest,16884.92400
1,18.0,male,33.770,1.0,0,southeast,1725.55230
2,28.0,male,33.000,3.0,0,southeast,4449.46200
3,33.0,male,22.705,0.0,0,northwest,21984.47061
4,32.0,male,28.880,0.0,0,northwest,3866.85520


In [31]:
# Split dataset

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

X.shape, X_train.shape

((1338, 6), (1070, 6))